# Hands On XGBoost
### Hands On Notebook

m|rig GmbH

Gesa Murphy

## Aufgabe 1: Laden der Python-Bibliotheken

In [ ]:
# Standard Python Bibliotheken
import pandas as pd
import numpy as np

# Bibliotheken für Machine Learning und Data Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from category_encoders.woe import WOEEncoder #wird nur im optionalen Log Reg Teil verwendet.

# xgboost Biblothek
import xgboost as xgb

# Bibliotheken für Visualisierung
from plot_functions import plot_features # Eigene Plot-Funktion
import matplotlib.pyplot as plt
import graphviz
from xgboost import to_graphviz
import seaborn as sns


# Bibliothek für SHAP values (XAI)
import shap

# Setze Anzeigeoptionen für Pandas
pd.set_option('display.max_columns', None)


## Aufgabe 2: Lade und analysiere den Datensatz "Taiwanese Credit Card Default Data"
Importiere die csv "default of credit card clients preprocessed.csv" als Data.Frame.

In [ ]:
# Lade csv als Data.Frame
df = pd.read_csv("default of credit card clients preprocessed v3.csv", sep=";")

# Die Merkmale des Datensatzes werden in ID (primary key), metrische und kategoriale erklärende Merkmale sowie die Zielvariable (abhängige Variable) unterteilt.
id_col = 'ID'
met_features = [col for col in df.drop(['ID', 'default'], axis=1).select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns.tolist() ]
cat_features = df.select_dtypes(include=['object', 'category']).columns.tolist() 
target_col = 'default'


### Merkmalserläuterungen
- ID: primary key
- LIMIT_BAL Kreditlimit 
- Sex: Geschlecht
- EDUCATION: höchster Bildungsabschluss
- MARRIAGE: Familienstand
- AGE: Alter in Jahren
- payment_status_mx: Verzugsstatus im x. Monat vor Ausfall
    - -1: Rechtzeitig alles bezahlt ('paid on time')
    - 0: Nur das nötigste gezahlt ('use of revolving credit')
    - 1-8: Anzahl Monate in Überziehung
    
- bill_amount_mx: Offener Betrag im x. Monat vor Ausfall
- payment_amount_mx: Bezahlter Betrag im x. Monat vor Ausfall
- default: Zielvariable (1 - Ausfall, 0 - kein Ausfall)


Aufgabe: 

Analysiere den Datensatz näher:
- Verwende die head-Methoden des DateFrames df
- Verwende die describe-Methode des DateFrames df auf den kategorialen Merkmalen verwende .astype("object") um alle kategorialen Features als solche zu betrachten.
- Verwende die describe-Methode des DateFrames df auf den numerischen Merkmalen inkl. percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]


In [ ]:
# Gibt die ersten 5 Zeilen des DataFrames aus
df.head()

In [ ]:
# Verteilung der kategorialen Merkmale
df[cat_features].describe(include='all')

In [ ]:
# Verteilung der metrischen Merkmale
df[met_features].describe(include='all', percentiles=[0.05, 0.25, 0.5, 0.75, 0.95])

Aufgabe: Erzeuge Grafiken zur Verteilung der einzelnen Merkmale.

Nutze hierzu die in plot_functions.py vordefinierten Funktion "plot_features".

In [ ]:
plot_features(df,target_col)

Achtung Korrelation: Sind viele Features miteinander korreliert, hat die logistische Regression manchmal Problem. (Zum Glück gibt es XGBoost)

In [ ]:
sns.heatmap(df[met_features].corr(method='pearson'))

Fragen Aufgabe 2:
- Zu welchen Merkmalen liegen Missings vor?
- Welche Merkmale sehen schon besonders trennscharf aus?
- Gehören die Missings eher zu den positiven oder negativen Merkmalen (Was ist ihre Ausfallrate im Vergleich zu den anderen Ausprägungen?)
- Welche Merkmale sind miteinander korreliert?

## Aufgabe 3: XGBoost
Trainiere ein xgboost-Modell. Speichere hierzu die erklärenden Variablen unter X_xgb und die Zielvariablen unter y ab.
Es gibt mind. zwei Möglichkeiten mit Objektvariablen umzugehen:
- Variante 1: kategorische Variablen mit OneHot-Encoding transformieren
- Variante 2: kategorische Variablen direkt verwenden

### 3.1 Variante 1: kategorische Variablen mit OneHot-Encoding transformieren

In [ ]:
# OneHot-Encoding der kategorialen Variablen inklusive Dummy-Variable für fehlende Werte
X_xgb_mD = pd.get_dummies(df.drop([id_col, target_col], axis = 1), columns=cat_features, dummy_na=True, drop_first=True, prefix=cat_features).copy()
y_xgb_mD =  df[target_col]

In [ ]:
# Train-Test-Split
X_train_xgb_mD, X_test_xgb_mD, y_train_xgb_mD, y_test_xgb_mD = train_test_split(
    X_xgb_mD, y_xgb_mD, test_size=0.2, random_state=42
)

In [ ]:
# Modelldesign und Training
xgb_model_mD = 

In [ ]:
# Modellevaluation
y_probs_mD = xgb_model_mD.predict_proba(X_test_xgb_mD)[:, 1]

gini = 2 * roc_auc_score(y_test_xgb_mD, y_probs_mD) - 1
print(f"Gini: {gini:.4f}")

### 3.2 Variante 2: kategorische Variablen direkt verwenden

In [ ]:
# Ersetze object column type mit category
X_xgb = pd.concat([df[met_features], df[cat_features].astype('category')], axis=1)
y_xgb =  df[target_col]

In [ ]:
# Train-Test-Split
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    X_xgb, y_xgb, test_size=0.2, random_state=42
)

In [ ]:
# Modelldesign und Training
xgb_model =

In [ ]:
# Modellevaluation
y_probs = xgb_model.predict_proba(X_test_xgb)[:, 1]

gini = 2 * roc_auc_score(y_test_xgb, y_probs) - 1
print(f"Gini: {gini:.4f}")

Frage Aufgabe 3.1 / 3.2:
- Was sind mögliche Vor- und Nachteile der beiden Varianten?

### 3.3 Xgboost-Modell verstehen


Aufgabe: Visualisiere den ersten Baum sowie die Bäume 10 / 50 /100 mit "to_graphviz".

Tipp: Verwende "size" um die Größe der Grafik anzupassen und rankdir='LR' um die Ausrichtung zu ändern.

Erklärung der Darstellung: Die Darstellung von Bäumen mit to_graphviz und plot_tree von XGB ist meiner Meinung nach noch nicht ideal und etwas verwirrend. Abzweigungen mit kategorischen Merkmalen zeigen zusätzlich zur enthaltenen Menge noch einen Wert für gain und cover an. Metrische Merkmale nicht.

Hinweis: Die Leaf Values sind Log Odds. Wendet man auf sie die Sigmoid Funktion an, erhält man Wahrscheinlichkeiten zwischen 0 und 1. Leaf Werte kleiner als 0 führen zu Wahrscheinlichkeiten unter 50%. Je höher der Leaf Value, für desto wahrscheinlicher wird ein Ausfall gehalten.

In [ ]:
# Darstellung des ersten Baumes

In [ ]:
# Darstellung des 10./20./100. Baumes

Aufgabe: Visualisiere die Feature Importance nach "gain". Du kannst auch die Importance mit "cover" oder "weight" betrachten.

Aufgabe 3.3:
- Welche Auffälligkeiten gibt es beim Vergleich der Bäume mit unterschiedlichen Iterationsnummern?
- Plausiblisiere die Feature Importance fachlich.

Antwort 3.3:
- Die Absolutwerte in den Leafs nehmen tendenziell ab.
- Aktuellere Features haben einen höheren Impact auf das Modell. Payment_status ist eng mit dem Ausfallkriterium verbunden und hat daher einen höheren Einfluss.

### 3.4 xgboost-Modell optimieren

Baumstruktur:


| Parameter           | Beschreibung                                                   | Typischer Bereich | Default |
|---------------------|------------------------------------------------------------------|-------------------|---------|
| `n_estimators`      | Anzahl der Bäume (Boosting-Runden)                              | 50 – 1000         | `100`   |
| `max_depth`         | Maximale Tiefe der Bäume                                        | 2 – 15            | `6`     |
| `min_child_weight`  | Minimale Summe der Gewichte pro Blattknoten                    | 1 – 10            | `1`     |
| `subsample`         | Anteil der Trainingsdaten, die pro Baum verwendet werden       | 0.5 – 1           | `1.0`   |
| `colsample_bytree`  | Anteil der Merkmale, die pro Baum verwendet werden             | 0.3 – 1           | `1.0`   |
| `colsample_bylevel` | Anteil der Merkmale pro Level (Tiefe)                          | 0.3 – 1           | `1.0`   |
| `colsample_bynode`  | Anteil der Merkmale pro Knoten                                 | 0.3 – 1           | `1.0`   |
| `scale_pos_weight`  | Gewichtung der positiven Klasse (bei Ungleichgewicht)          | 1 – 100 (je nach Unbalance) | `1.0` |

Lernrate und Regularisierung:

| Parameter       | Beschreibung                                               | Typischer Bereich | Default |
|-----------------|------------------------------------------------------------|-------------------|---------|
| `learning_rate` | Lernrate / Schrittweite des Boostings                      | 0.01 – 0.3        | `0.3`   |
| `gamma`         | Mindestverlustreduktion für einen Split (`min_split_loss`) | 0 – 10            | `0.0`   |
| `reg_alpha`     | L1-Regularisierung (Lasso) auf Blattwerte                  | 0 – 10            | `0.0`   |
| `reg_lambda`    | L2-Regularisierung (Ridge) auf Blattwerte                  | 0 – 10            | `1.0`   |

Es folgt Code für ein einfaches Gridsearch. Mit den vorgeschlagenen Parametern dauert es etwa 2 Minuten. Versuche den Code anzupassen (passe dazu das param_grid an) und auf ein noch besseres Ergebnis zu kommen.

Aber je mehr Parameter oder Werte du mit reinnimmst, desto länger läuft der Code!

In [ ]:
# Basis-Modell für Grid Search
xgb_model_GS = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    enable_categorical=True 
)

# Parameter-Grid definieren - hier nach Bedarf Änderungen vornehmen
param_grid = {
    'n_estimators': [250], # [100, 250, 500]
    'max_depth': [ 2, 3], #[3,4, 5,6 7],
    'gamma': [1, 2], #  [0, 1, 2, 3, 4, 5],
    'colsample_bytree': [0.8, 1.0], # [0.8],
    'learning_rate': [0.1, 0.1], # [0.01, 0.05, 0.1],
    'reg_alpha': [2, 3], # [0, 1, 2, 3, 4, 5],
    'reg_lambda': [1, 2] #[0, 3, 5] 
}

# Grid Search konfigurieren
grid_search = GridSearchCV(
    estimator=xgb_model_GS,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=3, # Cross-Validation mit 3 Folds
    verbose=2, # detaillierter Output
    n_jobs=-1 # alle verfügbaren Kerne nutzen 
)

# Training 100 fits < 1 min
grid_search.fit(X_train_xgb, y_train_xgb)

# Beste Parameter & Genauigkeit
print("Beste Parameter:", grid_search.best_params_)
print("Beste CrossValidation-Genauigkeit:", grid_search.best_score_)

# Test-Performance
best_model = grid_search.best_estimator_
y_probs = best_model.predict_proba(X_test_xgb)[:, 1]

gini = 2 * roc_auc_score(y_test_xgb, y_probs) - 1
print(f"Gini: {gini:.4f}")

Schau dir das best_model einmal an:

In [ ]:
best_model

## Aufgabe 4: Erklärbarkeit des Modells mit Shapley values.

### XGBoost

Verwende das best_model. Es sollte vermutlich kategorische Merkmale verwendet haben.

In [ ]:
# SHAP Werte berechnen
explainer_xgb = shap.Explainer(best_model)
shap_values_xgb = explainer_xgb(X_test_xgb)

Betrachte die Feature Importance mit SHAP über den summary_plot (plot_type='bar'). Vergleiche mit der feature_importance die direkt von XGBoost kam.

Betrachte den Einfluss der SHAP Value nach Feature Werten über den beeswarm Plot:

Vergleiche den Beeswarm Plot des best_model mit dem Model mit dummies:

In [ ]:
explainer_xgb_mD = shap.Explainer(xgb_model_mD)
shap_values_xgb_mD = explainer_xgb_mD(X_test_xgb_mD)

In [ ]:
shap.plots.beeswarm(shap_values_xgb_mD)

Betrachte die Verteilung der Shap Values bei einem einzelnen Feature, dem mit der größten Feature Importance: (zurück zum Best Model):

In [ ]:

shap.plots.scatter(shap_values_xgb[:, 'ersetze hier den namen des features mit höchster importance'],color =y_test_xgb.to_numpy())


## Aufgabe 5: Optional: Logistische Regression

Dieser Teil des Notebooks ist optional. Wir haben allen Code hier drin gelassen, da wir mit diesem Teil nur aufzeigen wollen, was bei logisitischer Regression, die in Banken üblicherweise für die Schätzung eines PD Modells genutzt wird, alles extra getan werden muss. Im Gegensatz zu XGBoost können wir die Daten nicht einfach in ein einfaches XGB Modell reinnehmen, sondern müssen mit Missings und anderen Problemen vorher umgehen:


### 5.1 Einfaches Logistisches Regressionsmodell
Baue zunächst ein logistisches Regressionsmodell mit einem Minimum an Datenvorbereitung (alle kategorischen Feature in dummy Variablen umwandeln).

In [ ]:
# OneHot-Encoding der kategorialen Variablen inklusive Dummy-Variable für fehlende Werte
X_lr_simple = pd.get_dummies(df.drop([id_col, target_col], axis = 1), columns=cat_features, drop_first=True, prefix=cat_features).copy()
y_lr_simple =  df[target_col]

In [ ]:

# Train-Test-Split
X_slr_train, X_slr_test, y_slr_train, y_slr_test = train_test_split(
    X_lr_simple,y_lr_simple, test_size=0.2, random_state=42
)

In [ ]:
# Modelldesign und Training
lr = LogisticRegression(max_iter=1000)
lr.fit(X_slr_train, y_slr_train)

Dieser Fehler kommt, weil na enthalten sind. Also: ganz so einfach geht es nicht. Wir können die nas ersetzen, z.B. durch den Median oder Mean:

In [ ]:
for feature in met_features:
    X_lr_simple[feature] = X_lr_simple[feature].fillna(X_lr_simple[feature].mean())

Wiederhole Train test split und Log Reg

In [ ]:
# Train-Test-Split
X_slr_train, X_slr_test, y_slr_train, y_slr_test = train_test_split(
    X_lr_simple,y_lr_simple, test_size=0.2, random_state=42
)

In [ ]:
# Modelldesign und Training
lr = LogisticRegression()
lr.fit(X_slr_train, y_slr_train)

Diesmal sollte eine Warnung gekommen sein, dass das Modell nicht konvergiert ist. (viele korrelierende Variablen). Man bekommt trotzdem eine Prediction (siehe nächstes Codefeld), aber eben nicht die beste.
Es gibt viele Lösungen, z.B. 
- max_iter hochsetzen
- korrelierende Variablen rausschmeißen (z.B. nur 1. Monat vor Ausfall benutzen (payment_status_m1, bill_amount_m1,..))
- Alle features skalieren mit
X_scaled = scaler.fit_transform(X_lr_simple)
X_slr_train, X_slr_test, y_slr_train, y_slr_test = train_test_split(
    X_scaled,y_lr_simple, test_size=0.2, random_state=42
)

Oder aber, wie in 5.2, mehr Datenaufbereitung betreiben und alle Merkmale gruppieren.

In [ ]:
# Modellevaluation
y_probs = lr.predict_proba(X_slr_test)[:, 1]

gini = 2 * roc_auc_score(y_slr_test, y_probs) - 1
print(f"Gini: {gini:.4f}")

### 5.2 Logistisches Regressionsmodell mit Datenaufbereitung
Baue ein logistisches Regressionsmodell mit einer ausführlicheren Datenaufbereitung. Dies entspricht sicher noch lange nicht einer vollständigen PD Modellierung, aber die Ergebnisse kommen dem schon näher.

Die Features werden alle gruppiert um dann später mit einem WOE Encoder einen bestimmten neuen Wert zu erhalten, der dann als Feature eingeht statt den Originalfeatures.
Die Behandlung von Missings wird mit .fillna(Wert_den_missing_erhalten_sollen) geregelt, also Missings werden nicht ausgeschlossen, sondern (zunächst) durch passende Kategorien (others) oder Mittlere Werte (bei good/bad/very bad ist bad der mittlere Wert). Falls ihr an dieser Stelle noch etwas Zeit habt, könnt ihr ja mal probieren, wieviel besser oder schlechter das Modell wird, wenn man die .fillna der payment status features auf good oder very bad setzt.

In [ ]:
df_lr = df.copy()

# Gruppiere Merkmale

# SEX, EDUCATION, MARRIAGE
df_lr['grp_SEX'] = df_lr['SEX'] # in einem richtigen PD Modell würde normalerweise das Geschlecht nicht eingehen.

df_lr['grp_EDUCATION'] = df_lr['EDUCATION'].replace({
    'graduate school': 'higher',
    'university': 'higher',
    'high school': 'medium and others',
    'others': 'medium and others'
}).fillna('medium and others') # fehlende Werte werden der Gruppe 'medium and others' zugeordnet

df_lr['grp_MARRIAGE'] = df_lr['MARRIAGE'].replace({
    'married': 'married',
    'single': 'single and others',
    'others': 'single and others'
}).fillna('single and others') # fehlende Werte werden der Gruppe 'single and others' zugeordnet

# Gruppiere numerische Merkmale
df_lr['grp_AGE'] = pd.cut(df_lr['AGE'], bins=[0, 25, 40, 50, 60, np.inf])

df_lr['grp_LIMIT_BAL'] = pd.cut(df_lr['LIMIT_BAL'], bins=[0, 25000, 50000, 100000, 150000, 300000, np.inf])

for feature in df_lr.columns:
    if feature.startswith('bill_amount'):
        df_lr['grp_' + feature] = pd.cut(df_lr[feature], bins=[-np.inf, 50000, np.inf])
    if feature.startswith('payment_amount'):
        df_lr['grp_' + feature] = pd.cut(df_lr[feature], bins=[0, 1000, 3000, 5000, 10000, np.inf])
    if feature.startswith('payment_status'): # Fokus auf payment_status Merkmale
        df_lr['grp_' + feature] = df_lr[feature].replace({
            -1: 'good',
            0: 'good',
            1: 'bad',
            2: 'very bad',
            3: 'very bad',
            4: 'very bad',
            5: 'very bad',
            6: 'very bad',
            7: 'very bad',
            8: 'very bad'
        }).fillna('bad')        # fehlende Werte der mittleren Kategorie zuordnen

# Reduziere Spalten auf gruppierte Merkmale
df_lr_grp = df_lr.loc[:, df_lr.columns.str.startswith('grp_')]


In [ ]:
# Train-Test-Split
X_lr_train, X_lr_test, y_lr_train, y_lr_test = train_test_split(
        df_lr_grp, df_lr[target_col], test_size=0.2, random_state=42
)

In [ ]:
# Modelldesign und Training
# Das Logistische Regressionsmodell wird in einer Pipeline zusammen mit dem WOE-Encoder trainiert.
pipeline = Pipeline(steps=[
    ("woe", WOEEncoder(
        cols=df_lr_grp.columns.tolist(),
        handle_missing="value",
        handle_unknown="value",
        randomized=False
    )),
    ("logit", LogisticRegression(max_iter=1000))
])

pipeline.fit(X_lr_train, y_lr_train)

In [ ]:
# Modellevaluation
y_probs = pipeline.predict_proba(X_lr_test)[:, 1]

gini = 2 * roc_auc_score(y_lr_test, y_probs) - 1
print(f"Gini: {gini:.4f}")

### SHAP Plots for Log REG

Hier sind die SHAP-Plots nochmal für die logistische Regression. Man sieht deutlich, dass die logistische Regression lineare Zusammenhänge abbildet: Die Beiträge der Merkmale sind konsistent und richten sich direkt nach den Koeffizienten.
Im Gegensatz dazu zeigt XGBoost ein deutlich komplexeres Bild: Die SHAP-Werte für ein und dasselbe Merkmal variieren stark zwischen Beobachtungen, da XGBoost nichtlineare Beziehungen und Wechselwirkungen zwischen Variablen modelliert. Dadurch entstehen individuelle, kontextabhängige Erklärungen.

In [ ]:
# SHAP Werte berechnen aus der Logistischen Regression
X_train_transformed = pipeline.named_steps['woe'].transform(X_lr_train)
X_test_transformed = pipeline.named_steps['woe'].transform(X_lr_test)

explainer_lr = shap.Explainer(pipeline.named_steps['logit'], X_train_transformed)
shap_values_lr = explainer_lr(X_test_transformed)


In [ ]:

shap.plots.scatter(shap_values_lr[:, 'grp_payment_status_m1'],color =y_test_xgb.to_numpy())

In [ ]:
# Zum Vergleich noch einmal XGBoost
shap.plots.scatter(shap_values_xgb[:, 'payment_status_m1'],color =y_test_xgb.to_numpy())

In [ ]:
shap.plots.beeswarm(shap_values_lr)

In [ ]:
# Zum Vergleich noch einmal XGBoost
shap.plots.beeswarm(shap_values_xgb)